Considere a base de Dados Trip Advisor Hotel Reviews:  

https://www.kaggle.com/datasets/andrewmvd/trip-advisor-hotel-reviews
1) Aplicar a limpeza da base de dados: Stopwords, caracteres especiais e stemming;
2) Transformar a base de dados em atributos numéricos: abordagens `CountVectorizer` e normalização `TfidfTransformer`;
3) Avaliar com os classificadores árvore de decisão e random forest

  

## Importando bibliotecas e dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re 
import nltk 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import (recall_score,
                             accuracy_score,
                             precision_score,
                             f1_score)
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Download dos itens necessários do NLTK
nltk.download('stopwords')
nltk.download('rslp')
nltk.download('punkt')

In [ ]:
df = pd.read_csv("../Atividades/dados/tripadvisor_hotel_reviews.csv")
df.head()

## Fazendo os estudos e limpezas iniciais na base

In [ ]:
df.info()

In [ ]:
df.shape

Podemos perceber que não existem valores faltantes na base.

In [ ]:
df['Rating'].value_counts()

In [ ]:
# Redução do número de labels em 'class'
def change_rating(x):
  if x > 3:
    return "Positive Rating"
  return "Negative Rating"

In [ ]:
# ALtera na Base de Dados (Dataframe)
df['classe'] = df['Rating'].apply(change_rating)
df.drop(columns='Rating', inplace=True)
df.head()

In [ ]:
df['classe'].value_counts()

## 1) Aplicar a limpeza da base de dados: Stopwords, caracteres especiais e stemming;

In [ ]:
# Remoção de duplicados
print(df.shape)
df.drop_duplicates(['Review'], inplace=True)
print(df.shape)

Nenhum valor duplicado foi encontrado. 

In [ ]:
# Remoção de Stopwords

def Remover_StopWords( Texto ):

    Lista_Palavras = Texto.split()

    # Texto sem as stopwords
    nova_frase = ''

    for word in Lista_Palavras:
        if word not in nltk.corpus.stopwords.words('english'):
            nova_frase = nova_frase + ' ' + word

    return nova_frase

df['Review'] = df['Review'].apply(Remover_StopWords)

df.head()

In [ ]:
# Limpeza de caracteres especiais, links e números

def Limpeza_Dados( Texto ):
    # Remoção de links
    etapa_01 = re.sub('www\S+', '', Texto)

    # Transformar para minúsculo
    etapa_02 = etapa_01.lower()

    # Remoção de carcateres epeciais
    etapa_03 = re.sub(r'[!~@#$%^&*()+=|{}[\]:;<.>?/\'\\",-]', '', etapa_02)

    # Remoção de números
    etapa_04 = re.sub('[0-9]', '', etapa_03)

    return etapa_04 

# Limpeza em todas as amostras
df['Review'] = df['Review'].apply(Limpeza_Dados)

df.head()

In [ ]:
# Transformação em Radical (Stemming)

Stem = nltk.stem.RSLPStemmer()

def Extrair_Radical( Texto ):

    Lista_Palavras = Texto.split()
    nova_frase = ''

    for word in Lista_Palavras:
        radical = Stem.stem(word)
        nova_frase = nova_frase + ' ' + radical
    
    return nova_frase


df['Review'] = df['Review'].apply(Extrair_Radical)
df.head()

## 2) Transformar a base de dados em atributos numéricos: abordagens `CountVectorizer` e normalização `TfidfTransformer`;

In [ ]:
df.head()

In [ ]:
# Separar entre previsores e classe
X = df['Review']
y = df['classe']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3)

In [ ]:
# Aplicar a trasnformação com CountVectorizer()
vect = CountVectorizer()

vect.fit(X_train)
X_train_vect = vect.transform(X_train)
X_test_vect = vect.transform(X_test)

In [ ]:
Tfidf = TfidfTransformer()
X_train_vect = Tfidf.fit_transform(X_train_vect)
X_test_vect = Tfidf.fit_transform(X_test_vect)

In [ ]:
print(X_train_vect.shape)
print(X_test_vect.shape)
print(y_train.shape)
print(y_test.shape)

## 3) Avaliar com os classificadores árvore de decisão e random forest

In [ ]:
# Instanciando o modelo
decTree = DecisionTreeClassifier()

# Treinamento
decTree.fit(X_train_vect, y_train)
decTree_pred = decTree.predict(X_test_vect)

In [ ]:
# Instanciando o modelo
ranForest = RandomForestClassifier()

# Treinamento
ranForest.fit(X_train_vect, y_train)
ranForest_pred = ranForest.predict(X_test_vect)

In [ ]:
cm = confusion_matrix(y_test, decTree_pred)
ConfusionMatrixDisplay(cm).plot()
plt.title("Classificador Decision Tree")
plt.show()
cm = confusion_matrix(y_test, ranForest_pred)
ConfusionMatrixDisplay(cm).plot()
plt.title("Classificador Random Forest")
plt.show()

In [ ]:
# Avaliacao

print("Decision Tree Classifier")
print( classification_report( y_test, decTree_pred ) )
print("Random Forest Classifier")
print( classification_report( y_test, ranForest_pred ) )

Portanto, conclui-se que o modelo Random Forest possui uma aderência melhor à esta base de dados.   
porém, devido ao desbalanceamento das amostras (~75% de classificações positivas), o modelo tem um score melhor quando tentamos predizer classificações positivas. 